In [38]:
!pip install tensorflow

In [50]:
import cv2
import tensorflow as tf

In [51]:
from IPython.display import display, clear_output

In [52]:
from PIL import Image

In [53]:
import numpy as np

In [54]:
model = tf.saved_model.load('model.savedmodel')
infer = model.signatures["serving_default"]

In [55]:
class_names = ['Normal','Fire']

In [56]:
cap = cv2.VideoCapture('http://172.25.216.246:8080/?action=stream')

In [57]:
frame_count=0
label="Loading..."
predicted_class = "Normal"
probability = 0.0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Convert BGR to RGB for Jupyter display
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame)

    clear_output(wait=True)
    display(img)

    frame_count+=1
    if frame_count % 100 ==0:
        resized_frame = cv2.resize(frame, (224, 224))
        normalized_frame = resized_frame.astype(np.float32) / 255.0
        input_tensor = tf.convert_to_tensor([normalized_frame])
        # Run inference
        # output = infer(input_tensor)
        output = infer(sequential_1_input=input_tensor)
        prediction = list(output.values())[0].numpy()[0]
        predicted_class = np.argmax(prediction)
        probability = prediction[predicted_class]

    # print("Prediction Array:", prediction)
    # print(f"Predicted Class: {class_names[predicted_class]}, Probability: {probability:.2f}")
    label = f"Class: {predicted_class}, Probability: {probability:.2f}"
    # ✅ Draw prediction text on the video frame
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 0, 255), 2, cv2.LINE_AA)

    # Alert only if Fire/Smoke detected with probability > 70%
    if probability > 0.7 and class_names[predicted_class] in ['Fire']:
        print("\n********** ALERT! **********")
        print(f"{class_names[predicted_class]} detected with probability {probability * 100:.2f}%")
        print("****************************\n")
    elif probability > 0.5 and class_names[predicted_class] in ['Normal']:
        print("\n********************")
        print(f"{class_names[predicted_class]} detected with probability {probability * 100:.2f}%")
        print("****************************\n")

# cap.release()

In [47]:
# print(infer.structured_input_signature)